In [1]:
import pandas as pd
import random, pickle

- définir les constantes
  - *million*, *debug*
  - facteurs de correction des fréquences :
    - *cFreqForme* => fréquence de la forme individuelle
    - *cFreqCases* => fréquence du lexème (Sg ou Pl)
    - *cFreqNull* => fréquence minimale

In [2]:
million=1000000
debug=False
cFreqForme=1000000
cFreqCases=1000
cFreqNull=1

- définir 
 - la taille des échantillons *nbMillions*
 - le nombre d'échantillons *nbSamples*
 - la référence du tirage

In [3]:
nbMillions=1
nbSamples=50
tiragePrefix="MGC-160801-0"
echantillon="-%dMo"%nbMillions

- choisir le type de fréquences à utiliser
 - fréquences brutes
 - fréquences ajustées

In [4]:
freqsBrutes=False

- lire le tableau de Uppsala dans un DataFrame
- renommer le Inst2Pl pour éliminer l'espace final en trop

In [5]:
pdRussianFreqs=pd.read_csv("/Users/gilles/ownCloud/Recherche/Boye/Russian-Data/Uppsala-Nouns.csv",sep=",",encoding="utf8")
pdRussianFreqs=pdRussianFreqs.rename(columns = {'Inst2Pl ':'Inst2Pl'})
#pdRussianFreqs

- lister les colonnes
 - *lstTitres* : toutes les colonnes
 - *lstCases* : toutes les cases
 - *lstCasesMarginales*: les cases exceptionnelles
 - *lstCasesPrincipales*: les cases centrales

In [6]:
lstTitres=pdRussianFreqs.columns.tolist()
if debug: print lstTitres

lstCases=lstTitres[6:-4]
if debug: print  len(lstCases), lstCases

if debug: print pdRussianFreqs[lstCases].sum()

lstCasesMarginales=lstCases[2:3]+lstCases[4:5]+lstCases[7:8]+lstCases[9:11]+lstCases[12:13]+lstCases[14:16]+lstCases[17:18]+lstCases[20:21]
if debug: print lstCasesMarginales

lstCasesPrincipales=list(set(lstCases)-set(lstCasesMarginales))
if debug: print lstCasesPrincipales

### Calculer la fréquence ajustée
- séparer les singuliers des pluriels qui ont chacun une fréquence pour le lexème
- la fréquence ajustée est calculée avec trois termes :
  1. fréquence de la forme\*facteur forme
  1. fréquence du lexème (Sg ou Pl)\*fréquence case\*facteur lexème
  1. constante de fréquence minimale

In [7]:
def freqsAdjust(row):
#    case=row["case"].strip()
    if case.endswith("Sg") or case.endswith("Sg2") or case=="Vocative":
        lexFreq=row["Sg"]
    elif case.endswith("Pl") or case.endswith("Pl2"):
        lexFreq=row["Pl"]
    else:
        print "PB avec l'identité de la case :", [row["case"]]
    freqAdjusted=cFreqForme*row["rawFreq"]+cFreqCases*caseFreq[case]*lexFreq+cFreqNull
    return int(freqAdjusted)

### Calcul des fréquences de cases
- *caseFreqs* note le nombre d'occurrences de chaque case (pd.Series)
- *casesOcc* note le nombre d'occurrences totales (int)
- *caseFreq* note la fréquence de chaque case (dict)


In [8]:
caseFreqs=pd.DataFrame(pdRussianFreqs[lstCases].sum(),columns=["caseOcc"])
#caseFreqs

In [9]:
casesOcc=caseFreqs["caseOcc"].sum()
caseFreqs["caseFreq"]=caseFreqs["caseOcc"]/casesOcc
caseFreq=caseFreqs["caseFreq"].to_dict()
#caseFreq

- faire la liste des cases pour Sg et pour Pl pour gérer les fréquences de lexème séparément

In [10]:
lstCasesSg=[case for case in lstCases if not "Pl" in case]
lstCasesPl=[case for case in lstCases if "Pl" in case]
print lstCasesSg,lstCasesPl

[u'NomSg', u'AccSg', u'Ref to NomSg', u'GenSg', u'Gen2Sg', u'DatSg', u'InstSg', u'InstSg2', u'LocSg', u'Loc2Sg', u'Vocative'] [u'NomPl', u'Nom2Pl', u'AccPl', u'Ref to Nom1Pl', u'Acc2Pl', u'GenPl', u'Gen2Pl', u'DatPl', u'InstPl', 'Inst2Pl', u'LocPl']


- calculer les fréquences de cases séparément pour Sg et Pl

In [11]:
casesOccSg=caseFreqs[caseFreqs.index.isin(lstCasesSg)]["caseOcc"].sum()
casesOccPl=caseFreqs[caseFreqs.index.isin(lstCasesPl)]["caseOcc"].sum()
caseFreqs.loc[caseFreqs.index.isin(lstCasesSg),"caseFreq"]=caseFreqs["caseOcc"]/casesOccSg
caseFreqs.loc[caseFreqs.index.isin(lstCasesPl),"caseFreq"]=caseFreqs["caseOcc"]/casesOccPl
#caseFreqs[caseFreqs[lstCasesPl]]["caseFreq"]=caseFreqs["caseOcc"]/casesOccPl
caseFreq=caseFreqs["caseFreq"].to_dict()
caseFreq

{u'Acc2Pl': 7.381054309797611e-05,
 u'AccPl': 0.015736407788488508,
 u'AccSg': 0.067703990206341033,
 u'DatPl': 0.04810971199126083,
 u'DatSg': 0.053705445061476502,
 u'Gen2Pl': 0.0038824345669535437,
 u'Gen2Sg': 0.0010704421931645464,
 u'GenPl': 0.35477775645473197,
 u'GenSg': 0.2826617934933674,
 'Inst2Pl': 0.00036905271548988059,
 u'InstPl': 0.086328811207392858,
 u'InstSg': 0.092590292686100223,
 u'InstSg2': 0.00024838990117630362,
 u'Loc2Sg': 0.0075936341216755673,
 u'LocPl': 0.083494486352430577,
 u'LocSg': 0.11566098326916595,
 u'Nom2Pl': 0.00053143591030542807,
 u'NomPl': 0.36804889210374808,
 u'NomSg': 0.36018309884143851,
 u'Ref to Nom1Pl': 0.038647200366100296,
 u'Ref to NomSg': 0.018558274045029542,
 u'Vocative': 2.3656181064409866e-05}

## Mettre le tableau en forme de lexique
- décroiser le tableau en liste de formes
- ajouter une colonne avec la fréquence ajustée
- calculer les fréquences cumulées et le max pour le tirage aléatoire

### Formater le tableau
  - décroiser le tableau
  - renommer les colonnes => case, rawFreq

In [12]:
lexique=pd.melt(pdRussianFreqs[["Gloss"]+lstCases+["Sg","Pl"]],id_vars=["Gloss","Sg","Pl"])
lexique=lexique.rename(columns = {'variable':'case',"value":"rawFreq"})

### Ajouter les fréquences ajustées


In [13]:
lexique["adjFreq"]=lexique.apply(freqsAdjust,axis=1)
#lexique

### Ajouter les fréquences fréquences cumulées
- fréquences cumulées
- max


In [14]:
if freqsBrutes:
    lexique["freqcum"]=lexique.rawFreq.cumsum()
else:
    lexique["freqcum"]=lexique.adjFreq.cumsum()
nbOccurrences=lexique.freqcum.max()
nbOccurrences

236949648034

### Incrémentation des formes choisies par le tirage
- tri du tirage
- identification de la forme courante
  - incrémentation de la forme courante

In [15]:
def tirage2triage(tirage):    
    triage=sorted(tirage)
#    print triage[0:10]
    freqTop=0
    indexMin=0
    tirs={}

    for num,tir in enumerate(triage[:]):
        if tir > freqTop:
            indexMin=lexique[lexique["freqcum"]>=tir].iloc[0].name
            freqTop=lexique.ix[indexMin,'freqcum']
            tirs[indexMin]=0
        tirs[indexMin] += 1
        if num%500000==0:
            print num,
    print
    for indexNum in tirs:
        lexique.ix[indexNum,'tir1']+=tirs[indexNum]

- on ajoute 1 à l'élément aléatoire pour ne pas démarrer avec un élément de fréquence nulle.

In [16]:
def creationSample(numero):
    lexique["tir1"]=0
    tirage=[]
    for n in range(nbMillions*million):
        tirage.append(random.randrange(nbOccurrences)+1)
    tirage2triage(tirage)
    with open("/Users/gilles/ownCloud/Recherche/Boye/Russian-Data/"+tiragePrefix+echantillon+"-%d"%numero+'-Tirage.pkl', 'wb') as output:
        pickle.dump(lexique, output, pickle.HIGHEST_PROTOCOL)
    with open("/Users/gilles/ownCloud/Recherche/Boye/Russian-Data/"+tiragePrefix+echantillon+"-%d"%numero+'-Nums.pkl', 'wb') as output:
        pickle.dump(tirage, output, pickle.HIGHEST_PROTOCOL)

In [17]:
for sample in range(nbSamples):
    print sample
    creationSample(sample)

0
0 500000
1
0 500000
2
0 500000
3
0 500000
4
0 500000
5
0 500000
6
0 500000
7
0 500000
8
0 500000
9
0 500000
10
0 500000
11
0 500000
12
0 500000
13
0 500000
14
0 500000
15
0 500000
16
0 500000
17
0 500000
18
0 500000
19
0 500000
20
0 500000
21
0 500000
22
0 500000
23
0 500000
24
0 500000
25
0 500000
26
0 500000
27
0 500000
28
0 500000
29
0 500000
30
0 500000
31
0 500000
32
0 500000
33
0 500000
34
0 500000
35
0 500000
36
0 500000
37
0 500000
38
0 500000
39
0 500000
40
0 500000
41
0 500000
42
0 500000
43
0 500000
44
0 500000
45
0 500000
46
0 500000
47
0 500000
48
0 500000
49
0 500000
